In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *

In [7]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')

In [8]:
model_t = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(32, 64, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(3136, 1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [10]:
batch_size = 64
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


In [11]:
def criterion_p(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) + (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))

In [12]:
def criterion_base(op, y, model_p):
    return nn.CrossEntropyLoss(reduction='sum')(op, y)

In [13]:
loss_history_base = train_model(model_t, trainloader, criterion_base, optimizer = optim.Adam(list(model_t.parameters()), lr=3e-4), epochs=30)

KeyboardInterrupt: 

In [14]:
model_t_v2 = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [15]:
# train v2 with only 2 conv layers
loss_history_test = train_model(model_t_v2, trainloader, criterion_base, optimizer = optim.Adam(list(model_t_v2.parameters()), lr=3e-4), epochs=30)

[1,   300] loss: 97.289
[1,   600] loss: 79.455


KeyboardInterrupt: 

In [ ]:
# ! conda list
from sklearn.metrics import *
y_v1, pred_y_v1 = get_predictions_ys(model_t, testloader)
print("3-conv-layer net f1 score: " + str(f1_score(y_v1, pred_y_v1, average=None)))
print("3-conv-layer net accuracy: " + str(accuracy_score(y_v1, pred_y_v1)))

y_v2, pred_y_v2 = get_predictions_ys(model_t_v2, testloader)
print("2-conv-layer net f1 score: " + str(f1_score(y_v2, pred_y_v2, average=None)))
print("2-conv-layer net accuracy: " + str(accuracy_score(y_v2, pred_y_v2)))

In [ ]:
print("Stats for 3-conv layer")
print_stats(model_t)

print("Stats for 2-conv layer")
print_stats(model_t_v2)

In [22]:
model_autoencoded = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [23]:
# train v2 with only 2 conv layers
encoder_loss_history_test = train_model_self_reg(model_autoencoded, trainloader, criterion_base, epochs=30)

[1,   300] loss: 106.260
[1,   600] loss: 92.253
[2,   300] loss: 83.559
[2,   600] loss: 67.769
[3,   300] loss: 64.695
[3,   600] loss: 63.473
[4,   300] loss: 69.090
[4,   600] loss: 50.230
[5,   300] loss: 46.203
[5,   600] loss: 45.633
[6,   300] loss: 42.268
[6,   600] loss: 39.803
[7,   300] loss: 33.558
[7,   600] loss: 28.456
[8,   300] loss: 24.236
[8,   600] loss: 15.206
[9,   300] loss: 32.541
[9,   600] loss: 19.956
[10,   300] loss: 9.630
[10,   600] loss: 18.619
[11,   300] loss: 11.118
[11,   600] loss: 23.304
[12,   300] loss: 2.913
[12,   600] loss: 18.697
[13,   300] loss: 5.816
[13,   600] loss: 15.566
[14,   300] loss: 4.608
[14,   600] loss: 1.940
[15,   300] loss: 6.537
[15,   600] loss: 3.817
[16,   300] loss: 2.475
[16,   600] loss: 6.695
[17,   300] loss: 12.743
[17,   600] loss: 5.929
[18,   300] loss: 5.560
[18,   600] loss: 9.534
[19,   300] loss: 9.308
[19,   600] loss: 6.872
[20,   300] loss: 1.696
[20,   600] loss: 14.989
[21,   300] loss: 2.355
[21,   6

In [26]:
print("Stats for encoded layer")
print_stats(model_autoencoded)

y_enc, pred_y_enc = get_predictions_ys(model_autoencoded, testloader)
print("encoded-layer net f1 score: " + str(f1_score(y_enc, pred_y_enc, average=None)))
print("encoded-layer net accuracy: " + str(accuracy_score(y_enc, pred_y_enc)))

y_v1, pred_y_v1 = get_predictions_ys(model_t, testloader)
print("3-conv-layer net f1 score: " + str(f1_score(y_v1, pred_y_v1, average=None)))
print("3-conv-layer net accuracy: " + str(accuracy_score(y_v1, pred_y_v1)))

y_v2, pred_y_v2 = get_predictions_ys(model_t_v2, testloader)
print("2-conv-layer net f1 score: " + str(f1_score(y_v2, pred_y_v2, average=None)))
print("2-conv-layer net accuracy: " + str(accuracy_score(y_v2, pred_y_v2)))

Stats for encoded layer
Mean
0.040562984
standard deviation
0.032107852
Sparsity
0.0015999022854130515
encoded-layer net f1 score: [0.71361502 0.78433367 0.58363731 0.49614644 0.60639361 0.55636896
 0.73207931 0.71525247 0.78912214 0.72831816]
encoded-layer net accuracy: 0.6701


NameError: name 'model_t' is not defined

In [27]:
encoder_loss_history__mse_test = train_model_self_reg(model_autoencoded, trainloader, criterion_base, epochs=30)

[1,   300] loss: 13.080
[1,   600] loss: 4.194
[2,   300] loss: 1.871
[2,   600] loss: 5.305
[3,   300] loss: 1.003
[3,   600] loss: 3.374
[4,   300] loss: 1.097
[4,   600] loss: 1.832
[5,   300] loss: 1.238
[5,   600] loss: 1.315
[6,   300] loss: 4.913
[6,   600] loss: 0.178
[7,   300] loss: 2.337
[7,   600] loss: 2.275
[8,   300] loss: 0.196
[8,   600] loss: 12.094
[9,   300] loss: 0.837
[9,   600] loss: 0.340
[10,   300] loss: 5.283
[10,   600] loss: 0.730
[11,   300] loss: 5.674
[11,   600] loss: 3.058
[12,   300] loss: 1.289
[12,   600] loss: 0.688
[13,   300] loss: 3.814
[13,   600] loss: 0.569
[14,   300] loss: 2.216
[14,   600] loss: 0.276
[15,   300] loss: 4.023
[15,   600] loss: 6.348
[16,   300] loss: 0.122
[16,   600] loss: 2.820
[17,   300] loss: 3.682
[17,   600] loss: 8.774
[18,   300] loss: 0.285
[18,   600] loss: 8.966
[19,   300] loss: 1.230
[19,   600] loss: 4.512
[20,   300] loss: 6.469
[20,   600] loss: 6.002
[21,   300] loss: 3.267
[21,   600] loss: 0.045
[22,   3